# Importing required Dependencies

In [1]:
import polars as pl
import pandas as pd
from flask import Flask, jsonify, request,make_response
import sqlite3
import requests
import math
import numpy as np
import time
import pytz
import random
import string
from datetime import datetime,timedelta

In [2]:
buisness_hours_df = pl.read_csv('buisness_hours.csv')
store_status_df = pl.read_csv('store_status.csv')
timezones_df = pl.read_csv('timezones.csv')

## Checking null values and filling them with default values

In [3]:
print(buisness_hours_df.head)
print(store_status_df.head)
print(timezones_df.head)

<bound method DataFrame.head of shape: (86_198, 4)
┌─────────────────────┬─────┬──────────────────┬────────────────┐
│ store_id            ┆ day ┆ start_time_local ┆ end_time_local │
│ ---                 ┆ --- ┆ ---              ┆ ---            │
│ i64                 ┆ i64 ┆ str              ┆ str            │
╞═════════════════════╪═════╪══════════════════╪════════════════╡
│ 1481966498820158979 ┆ 4   ┆ 00:00:00         ┆ 00:10:00       │
│ 1481966498820158979 ┆ 2   ┆ 00:00:00         ┆ 00:10:00       │
│ 1481966498820158979 ┆ 0   ┆ 00:00:00         ┆ 00:10:00       │
│ 1481966498820158979 ┆ 1   ┆ 00:00:00         ┆ 00:10:00       │
│ …                   ┆ …   ┆ …                ┆ …              │
│ 1774428314218854399 ┆ 6   ┆ 00:00:00         ┆ 23:59:59       │
│ 1774428314218854399 ┆ 3   ┆ 00:00:00         ┆ 23:59:59       │
│ 1774428314218854399 ┆ 2   ┆ 00:00:00         ┆ 23:59:59       │
│ 1774428314218854399 ┆ 1   ┆ 00:00:00         ┆ 23:59:59       │
└─────────────────────┴──

In [4]:

unique_store_ids_buisness_hours = buisness_hours_df.select('store_id').n_unique()
unique_store_ids_store_status = store_status_df.select('store_id').n_unique()
unique_store_ids_timezones = timezones_df.select('store_id').n_unique()

print('buisness_hours.csv',unique_store_ids_buisness_hours)
print('store_status.csv',unique_store_ids_store_status)
print('timezones.csv',unique_store_ids_timezones)

max_unique_store_ids = max(unique_store_ids_buisness_hours, unique_store_ids_store_status, unique_store_ids_timezones)

if max_unique_store_ids == unique_store_ids_buisness_hours:
    print("buisness_hours.csv has the highest number of unique store_id values.")
elif max_unique_store_ids == unique_store_ids_store_status:
    print("store_status.csv has the highest number of unique store_id values.")
else:
    print("timezones.csv has the highest number of unique store_id values.")

buisness_hours.csv 11116
store_status.csv 14092
timezones.csv 13559
store_status.csv has the highest number of unique store_id values.


In [ ]:
column_name = "end_time_local"

max_value = buisness_hours_df[column_name].max()


print(f"The maximum value in the '{column_name}' column is: {max_value}")

is_null_values = buisness_hours_df[column_name].is_null()

num_null_values = is_null_values.sum()

print(f"The number of null values in '{column_name}' column: {num_null_values}")

In [5]:
unique_store_ids = store_status_df['store_id'].unique()
# print(len(unique_store_ids))
for store_id in unique_store_ids:
    if store_id not in buisness_hours_df['store_id']:
#         print(f"store_id {store_id} is not in buisness_hours_df")
        for day in range(7):
            new_row = {'store_id': [store_id], 'day' :[day] ,'start_time_local' :['00:00:00'],'end_time_local':['23:59:59']}
            df_new_row = pl.DataFrame(new_row)
            buisness_hours_df = pl.concat([buisness_hours_df, df_new_row])
    if store_id not in timezones_df['store_id']:
#         print(f"store_id {store_id} is not in timezone_df")
        new_row = {'store_id': [store_id], 'timezone_str': ['America/Chicago']}
        df_new_row = pl.DataFrame(new_row)
        timezones_df = pl.concat([timezones_df, df_new_row])
# print(buisness_hours_df)
# print(timezones_df)

In [6]:
current_time = store_status_df['timestamp_utc'].max()

print("Maximum date in 'timestamp_utc' column:", current_time)

Maximum date in 'timestamp_utc' column: 2023-01-25 18:13:22.47922 UTC


## Connecting to Database 'Stores'  in Sqlite3

### Creating Tables and schema

In [7]:
# conn = sqlite3.connect("stores.db")
# cursor = conn.cursor()

# query_store_status = '''CREATE TABLE store_status (
#         store_id VARCHAR(255) ,
#         timestamp_utc TIMESTAMP NOT NULL,
#         status VARCHAR(10) NOT NULL
#     );'''

# query_business_hours='''
#     CREATE TABLE business_hours (
#         store_id VARCHAR(255),
#         day_of_week INTEGER NOT NULL,
#         start_time_local TIME,
#         end_time_local TIME
#     );
# '''

# query_store_timezones='''
#     CREATE TABLE store_timezones (
#         store_id VARCHAR(255) PRIMARY KEY,
#         timezone_str VARCHAR(255) NOT NULL
#     );'''

# query_report_status='''
#     CREATE TABLE report_status (
#         report_id VARCHAR PRIMARY KEY,
#         status VARCHAR NOT NULL
#     );'''
# queries = [query_store_status,query_business_hours,query_store_timezones,query_report_status]
# for i in range(len(queries)):
#     cursor.execute(queries[i])
# conn.commit()
# conn.close()

In [8]:

# conn = sqlite3.connect('stores.db')
# cursor = conn.cursor()

# for row in store_status_df.rows():
#     insert_query = 'INSERT INTO store_status (store_id, status, timestamp_utc) VALUES (?, ?, ?)'
#     cursor.execute(insert_query, row)

# conn.commit()
# cursor.close()
# conn.close()


In [9]:
# conn = sqlite3.connect('stores.db')
# cursor = conn.cursor()

# for row in timezones_df.rows():
#     insert_query = 'INSERT INTO store_timezones (store_id, timezone_str) VALUES (?,?)'
#     cursor.execute(insert_query, row)

# conn.commit()
# cursor.close()
# conn.close()


In [10]:
# conn = sqlite3.connect('stores.db')
# cursor = conn.cursor()

# for row in buisness_hours_df.rows():
#     insert_query = 'INSERT INTO business_hours (store_id, day_of_week,start_time_local,end_time_local) VALUES (?,?,?,?)'
#     cursor.execute(insert_query, row)

# conn.commit()
# cursor.close()
# conn.close()

In [11]:

conn = sqlite3.connect('stores.db')
cursor = conn.cursor()

cursor.execute('SELECT * FROM business_hours LIMIT 5;')
records = cursor.fetchall()
print("Total rows in business_hours are:  ", len(records))
print(records)

cursor.execute('SELECT * FROM store_status LIMIT 5;')
records = cursor.fetchall()
print("Total rows in store_status are:  ", len(records))
print(records)

cursor.execute('SELECT * FROM store_timezones LIMIT 5;')
records = cursor.fetchall()
print("Total rows in store_timezones are:  ", len(records))
print(records)

conn.commit()
cursor.close()
conn.close()

Total rows in business_hours are:   5
[('1481966498820158979', 4, '00:00:00', '00:10:00'), ('1481966498820158979', 2, '00:00:00', '00:10:00'), ('1481966498820158979', 0, '00:00:00', '00:10:00'), ('1481966498820158979', 1, '00:00:00', '00:10:00'), ('1481966498820158979', 5, '00:00:00', '00:10:00')]
Total rows in store_status are:   5
[('8419537941919820732', '2023-01-22 12:09:39.388884 UTC', 'active'), ('54515546588432327', '2023-01-24 09:06:42.605777 UTC', 'active'), ('8377465688456570187', '2023-01-24 09:07:26.441407 UTC', 'active'), ('5955337179846162144', '2023-01-24 09:08:07.634899 UTC', 'active'), ('1169347689335808384', '2023-01-24 09:08:18.436854 UTC', 'active')]
Total rows in store_timezones are:   5
[('8139926242460185114', 'Asia/Beirut'), ('5415949628544298339', 'America/Boise'), ('3408529570017053440', 'America/Denver'), ('9055649751952768824', 'America/Denver'), ('4428372089193592098', 'America/Denver')]


# Interpolation

In [12]:
def convert_time_to_utc(local_time_str, timezone_str):
    local_tz = pytz.timezone(timezone_str)
    local_time = datetime.strptime(local_time_str, "%H:%M:%S").time()

    current_date = datetime.now(local_tz).date()

    local_dt = datetime.combine(current_date, local_time)

    utc_dt = local_tz.localize(local_dt, is_dst=None).astimezone(pytz.utc)

    utc_time_str = utc_dt.strftime("%H:%M:%S")
    return utc_time_str

In [13]:
def get_day_of_week(time_str):
    time_obj = datetime.strptime(time_str, "%Y-%m-%d %H:%M:%S.%f UTC")

    day_of_week = time_obj.weekday()

    return day_of_week


In [14]:
print(unique_store_ids[2])

filtered_df = buisness_hours_df.filter(pl.col('store_id') == unique_store_ids[2])
filtered_store_df = store_status_df.filter(pl.col('store_id') == unique_store_ids[2])

print(filtered_df)
print(filtered_store_df)

2570905277901393
shape: (7, 4)
┌──────────────────┬─────┬──────────────────┬────────────────┐
│ store_id         ┆ day ┆ start_time_local ┆ end_time_local │
│ ---              ┆ --- ┆ ---              ┆ ---            │
│ i64              ┆ i64 ┆ str              ┆ str            │
╞══════════════════╪═════╪══════════════════╪════════════════╡
│ 2570905277901393 ┆ 2   ┆ 00:00:00         ┆ 23:59:00       │
│ 2570905277901393 ┆ 4   ┆ 00:00:00         ┆ 23:59:00       │
│ 2570905277901393 ┆ 0   ┆ 00:00:00         ┆ 23:59:00       │
│ 2570905277901393 ┆ 1   ┆ 00:00:00         ┆ 23:59:00       │
│ 2570905277901393 ┆ 5   ┆ 00:00:00         ┆ 23:59:00       │
│ 2570905277901393 ┆ 3   ┆ 00:00:00         ┆ 23:59:00       │
│ 2570905277901393 ┆ 6   ┆ 00:00:00         ┆ 23:59:00       │
└──────────────────┴─────┴──────────────────┴────────────────┘
shape: (244, 3)
┌──────────────────┬──────────┬────────────────────────────────┐
│ store_id         ┆ status   ┆ timestamp_utc                  │
│ --

In [15]:
# # # Filter 'timezones_df' for store_id=5
# # store_id_to_find = unique_store_ids[230]


# # filtered_status_df = store_status_df.filter(pl.col('store_id') == store_id_to_find)

# # filtered_buisness_hours_df = buisness_hours_df.filter(pl.col('store_id') == store_id_to_find)

# # filtered_timezones_df = timezones_df.filter(pl.col('store_id') == store_id_to_find)

# # timezones_str = filtered_timezones_df['timezone_str'].to_list()[0]
# # print(timezones_str)
# # print(filtered_buisness_hours_df)

# # print(current_time)

# max_time = current_time
# result_data=[]

# # for store_id in unique_store_ids:

# filtered_status_df = store_status_df.filter(pl.col('store_id') == store_id)

# filtered_df = buisness_hours_df.filter(buisness_hours_df['store_id'] == store_id)

# timezones_str = timezones_df.filter(pl.col('store_id') == store_id)['timezone_str'].to_list()[0]

# buisness_hour_store = filtered_df.with_columns(pl.lit(filtered_df['start_time_local'].apply(lambda x: convert_time_to_utc(x,timezones_str))).alias('start_time_utc')).with_columns(pl.lit(filtered_df['end_time_local'].apply(lambda x: convert_time_to_utc(x,timezones_str))).alias('end_time_utc')).drop(['start_time_local', 'end_time_local'])

# #     print(buisness_hour_store)

# current_times = filtered_status_df['timestamp_utc']
# for current_time in current_times:

#     day_of_week = get_day_of_week(current_time)

#     filtered_df = buisness_hour_store.filter(
#         (buisness_hour_store['store_id'] == store_id) &
#         (buisness_hour_store['day'] == day_of_week)
#     )

#     is_within_business_hours = any(
#         ((current_time >= filtered_df['start_time_utc']) &
#          (current_time <= filtered_df['end_time_utc'])) |
#         ((filtered_df['end_time_utc'] < filtered_df['start_time_utc']) &
#          ((current_time >= filtered_df['start_time_utc']) |
#           (current_time <= filtered_df['end_time_utc'])))
#     )

#     #     print(f"The given time {current_time} is within business hours: {is_within_business_hours}")
#     if is_within_business_hours != True :
#         filtered_status_df = filtered_status_df.filter(pl.col('timestamp_utc') != current_time)
#     #             print('data entry which is not in buisness hours have been removed.')

#     #     print(filtered_status_df)


In [16]:
result_data=[]

store_id = unique_store_ids[4]

filtered_status_df = store_status_df.filter(pl.col('store_id') == store_id)

filtered_df = buisness_hours_df.filter(buisness_hours_df['store_id'] == store_id)

timezones_str = timezones_df.filter(pl.col('store_id') == store_id)['timezone_str'].to_list()[0]

buisness_hour_store = filtered_df.with_columns(pl.lit(filtered_df['start_time_local'].apply(lambda x: convert_time_to_utc(x,timezones_str))).alias('start_time_utc')).with_columns(pl.lit(filtered_df['end_time_local'].apply(lambda x: convert_time_to_utc(x,timezones_str))).alias('end_time_utc')).drop(['start_time_local', 'end_time_local'])

#     print(buisness_hour_store)

current_times = filtered_status_df['timestamp_utc']
for current_time in current_times:

    day_of_week = get_day_of_week(current_time)

    filtered_df = buisness_hour_store.filter(
        (buisness_hour_store['store_id'] == store_id) &
        (buisness_hour_store['day'] == day_of_week)
    )

    is_within_business_hours = any(
        ((current_time >= filtered_df['start_time_utc']) &
         (current_time <= filtered_df['end_time_utc'])) |
        ((filtered_df['end_time_utc'] < filtered_df['start_time_utc']) &
         ((current_time >= filtered_df['start_time_utc']) |
          (current_time <= filtered_df['end_time_utc'])))
    )

    #     print(f"The given time {current_time} is within business hours: {is_within_business_hours}")
    if is_within_business_hours != True :
        filtered_status_df = filtered_status_df.filter(pl.col('timestamp_utc') != current_time)
    #             print('data entry which is not in buisness hours have been removed.')

    #     print(filtered_status_df)

sorted_df = filtered_status_df.sort('timestamp_utc')
#     print(sorted_df)

df = sorted_df

timestamp_list = df['timestamp_utc'].to_list()

pandas_df = pd.DataFrame({
    'store_id': df['store_id'].to_list(),
    'status': df['status'].to_list(),
    'timestamp_utc': timestamp_list
})

#     print(pandas_df)
df = pandas_df
df['timestamp_utc'] = pd.to_datetime(df['timestamp_utc'])

current_time_str = "2023-01-25 18:05:39.487728 UTC"
current_time = pd.to_datetime(current_time_str)

one_hour_before = current_time - pd.Timedelta(hours=1)

one_hour_df = df[
    (df['timestamp_utc'] >= one_hour_before) &
    (df['timestamp_utc'] <= current_time)
]

print(one_hour_df)

uptime_last_hour = 0
downtime_last_hour = 0
last_timestamp = None

for index, row in one_hour_df.iterrows():
    if last_timestamp is not None:
        time_difference = row['timestamp_utc'] - last_timestamp
        if row['status'] == 'active':
            uptime_last_hour += time_difference.total_seconds() / 60
        elif row['status'] == 'inactive':
            downtime_last_hour += time_difference.total_seconds() / 60
    last_timestamp = row['timestamp_utc']

print("Uptime Hours:", uptime_last_hour)
print("Downtime Hours:", downtime_last_hour)


one_day_before = current_time - pd.Timedelta(days=1)

one_day_df = df[(df['timestamp_utc'] >= one_day_before.strftime("%Y-%m-%d %H:%M:%S.%f UTC")) & 
                 (df['timestamp_utc'] <= current_time_str)]

uptime_last_day = 0
downtime_last_day = 0
last_timestamp = None

for index, row in one_day_df.iterrows():
    if last_timestamp is not None:
        time_difference = row['timestamp_utc'] - last_timestamp
        if row['status'] == 'active':
            uptime_last_day += time_difference.total_seconds() / 3600
        elif row['status'] == 'inactive':
            downtime_last_day += time_difference.total_seconds() / 3600
    last_timestamp = row['timestamp_utc']


print(one_day_df.head())

print("Uptime Hours:", uptime_last_day)
print("Downtime Hours:", downtime_last_day)


one_week_before = current_time - pd.Timedelta(weeks=1)

one_week_df = df[(df['timestamp_utc'] >= one_week_before.strftime("%Y-%m-%d %H:%M:%S.%f UTC")) & 
                 (df['timestamp_utc'] <= current_time_str)]
uptime_last_week = 0
downtime_last_week = 0
last_timestamp = None

for index, row in one_week_df.iterrows():
    if last_timestamp is not None:
        time_difference = row['timestamp_utc'] - last_timestamp
        if row['status'] == 'active':
            uptime_last_week += time_difference.total_seconds() / 3600
        elif row['status'] == 'inactive':
            downtime_last_week += time_difference.total_seconds() / 3600
    last_timestamp = row['timestamp_utc']

print("Uptime Hours:", uptime_last_week)
print("Downtime Hours:", downtime_last_week)

print(one_week_df)

result_data.append([
    store_id, uptime_last_hour, uptime_last_day, uptime_last_week,
    downtime_last_hour, downtime_last_day, downtime_last_week
])
print(result_data)


             store_id  status                    timestamp_utc
114  3225796639170927  active 2023-01-25 17:05:54.303842+00:00
115  3225796639170927  active 2023-01-25 18:03:12.935628+00:00
Uptime Hours: 57.31052976666667
Downtime Hours: 0
            store_id  status                    timestamp_utc
91  3225796639170927  active 2023-01-24 19:03:33.231540+00:00
92  3225796639170927  active 2023-01-24 20:03:20.821002+00:00
93  3225796639170927  active 2023-01-24 21:03:09.051003+00:00
94  3225796639170927  active 2023-01-24 22:03:48.661221+00:00
95  3225796639170927  active 2023-01-24 22:06:10.438414+00:00
Uptime Hours: 22.994362246666668
Downtime Hours: 0
Uptime Hours: 167.7795658233335
Downtime Hours: 0
             store_id  status                    timestamp_utc
5    3225796639170927  active 2023-01-18 18:16:26.498664+00:00
6    3225796639170927  active 2023-01-18 19:40:41.384087+00:00
7    3225796639170927  active 2023-01-18 20:09:49.188924+00:00
8    3225796639170927  active 2023-0

In [17]:
result_df = pd.DataFrame(
    {'store_id': [row[0] for row in result_data],
     'uptime_last_hour(in minutes)': [row[1] for row in result_data],
     'uptime_last_day(in hours)': [row[2] for row in result_data],
     'uptime_last_week(in hours)': [row[3] for row in result_data],
     'downtime_last_hour(in minutes)': [row[4] for row in result_data],
     'downtime_last_day(in hours)': [row[5] for row in result_data],
     'downtime_last_week(in hours)': [row[6] for row in result_data]}
)

print(result_df)

           store_id  uptime_last_hour(in minutes)  uptime_last_day(in hours)  \
0  3225796639170927                      57.31053                  22.994362   

   uptime_last_week(in hours)  downtime_last_hour(in minutes)  \
0                  167.779566                               0   

   downtime_last_day(in hours)  downtime_last_week(in hours)  
0                            0                             0  


In [ ]:

app = Flask(__name__)

report_data = {}

def generate_report_id():
    return ''.join(random.choices(string.ascii_letters + string.digits, k=10))

@app.route('/trigger_report', methods=['GET'])
def trigger_report():
    report_id = generate_report_id()
    report_data[report_id] = {
        'status': 'Running',
        'csv_data': None
    }
    
    time.sleep(5)
    
    report_data[report_id]['status'] = 'Complete'
    report_data[report_id]['csv_data'] = result_df.to_csv(index=False)
    return jsonify({'report_id': report_id})

@app.route('/get_report', methods=['GET'])
def get_report():
    report_id = request.args.get('report_id')
    if report_id in report_data:
        status = report_data[report_id]['status']
        if status == 'Running':
            return jsonify({'status': 'Running'})
        elif status == 'Complete':
            csv_data = report_data[report_id]['csv_data']
            response = make_response(csv_data)
            response.headers["Content-Disposition"] = "attachment; filename=report.csv"
            response.headers["Content-type"] = "text/csv"
            return response
    else:
        return jsonify({'status': 'Invalid report_id'})

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [05/Aug/2023 17:40:21] "GET /trigger_report HTTP/1.1" 200 -
127.0.0.1 - - [05/Aug/2023 17:40:51] "GET /trigger_report HTTP/1.1" 200 -
127.0.0.1 - - [05/Aug/2023 17:41:15] "GET /get_report?report_id=F1BwbB1QxF HTTP/1.1" 200 -
127.0.0.1 - - [05/Aug/2023 17:42:59] "GET /trigger_report HTTP/1.1" 200 -
